In [1]:
!pip install -q langchain langchain-community langchain-groq chromadb sentence-transformers PyPDF2 unstructured tiktoken faiss-cpu gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 29.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 79.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 110.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.4/131.4 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 82.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
import PyPDF2
from typing import List, Dict, Tuple
import numpy as np
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_groq import ChatGroq
from langchain.schema import Document
import gradio as gr
from datetime import datetime
import re
from google.colab import drive

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
os.environ["GROQ_API_KEY"] = "API"

In [5]:
DOCS_DIR = "/content/drive/MyDrive/Legal"
CHUNK_SIZE = 1000
CHUNK_OVERLAP = 200
MODEL_NAME = "llama-3.3-70b-versatile"

In [6]:
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2",
    model_kwargs={'device': 'cuda'}
)

/tmp/ipython-input-1977789941.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/402 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
class EnhancedNepaliLegalRetriever:
    """Enhanced retriever with document priority system"""

    def __init__(self):
        self.document_priority = {
            # Highest Priority: Constitution
            "constitution": 1,

            # Second Priority: Core Legal Codes
            "मुलुकी_देवानी_संहिता": 2,
            "मुलुकी_फौजदारी_कार्यविधि_संहिता": 2,
            "मुलुकी_अपराध_संहिता": 2,
            "मुलुकी_देवानी_कार्यविधि_संहिता": 2,

            # Third Priority: Acts
            "ऐन": 3,
            "acts": 3,
            "additional-acts": 3,

            # Lowest Priority: Rules and Regulations
            "नियमावली": 4,
            "rules": 4,
            "regulation": 4
        }

        self.legal_categories = {
            "divorce": ["तलाक", "विवाह विच्छेद", "divorce", "separation", "वैवाहिक"],
            "custody": ["हिरासत", "custody", "child custody", "बच्चाको हिरासत", "संरक्षकत्व"],
            "financial_support": ["भरणपोषण", "financial support", "alimony", "maintenance", "आर्थिक सहायता"],
            "domestic_violence": ["घरेलु हिंसा", "domestic violence", "abuse", "हिंसा"],
            "property": ["सम्पत्ति", "property", "inheritance", "वंशाधिकार", "जग्गा"],
            "marriage": ["विवाह", "marriage", "wedding", "बिहे"],
            "criminal": ["फौजदारी", "criminal", "अपराध", "crime"],
            "civil": ["दीवानी", "civil", "नागरिक"]
        }

    def get_document_priority(self, source_path: str) -> int:
        """Get priority score for document (lower is higher priority)"""
        source_lower = source_path.lower()

        # Check for constitution first
        if "constitution" in source_lower or "संविधान" in source_lower:
            return 1

        # Check for core legal codes
        for code in ["मुलुकी_देवानी_संहिता", "मुलुकी_फौजदारी_कार्यविधि_संहिता",
                     "मुलुकी_अपराध_संहिता", "मुलुकी_देवानी_कार्यविधि_संहिता"]:
            if code in source_lower:
                return 2

        # Check for acts
        if "ऐन" in source_lower or "/acts/" in source_lower or "/additional-acts/" in source_lower:
            return 3

        # Check for rules and regulations
        if "नियमावली" in source_lower or "/rules" in source_lower or "/regulation" in source_lower:
            return 4

        return 5  # Default priority

    def identify_legal_categories(self, question: str) -> List[str]:
        """Identify multiple legal categories from question"""
        question_lower = question.lower()
        identified_categories = []

        for category, keywords in self.legal_categories.items():
            if any(keyword in question_lower for keyword in keywords):
                identified_categories.append(category)

        return identified_categories if identified_categories else ["general"]

    def retrieve_prioritized_documents(self, question: str, vectorstore, k: int = 10) -> List[Document]:
        """Retrieve documents with priority-based ranking"""
        if not vectorstore:
            return []

        # Get initial retrieval results
        retriever = vectorstore.as_retriever(
            search_type="similarity",
            search_kwargs={"k": k * 2}  # Get more initially for reranking
        )

        initial_docs = retriever.get_relevant_documents(question)

        if not initial_docs:
            return []

        # Add priority scores and rerank
        scored_docs = []
        for doc in initial_docs:
            priority_score = self.get_document_priority(doc.metadata.get('source', ''))
            scored_docs.append((doc, priority_score))

        # Sort by priority (lower score = higher priority)
        scored_docs.sort(key=lambda x: x[1])

        # Return top k documents
        return [doc for doc, _ in scored_docs[:k]]

In [9]:
class AdvancedNepaliLegalAssistant:
    def __init__(self, vectorstore, llm):
        self.vectorstore = vectorstore
        self.llm = llm
        self.retriever = EnhancedNepaliLegalRetriever()

    def get_comprehensive_response(self, question: str) -> str:
        """Get comprehensive response with enhanced retrieval"""
        if not self.vectorstore:
            return "❌ Legal documents not loaded."

        try:
            # Identify legal categories
            categories = self.retriever.identify_legal_categories(question)

            # Get prioritized documents
            relevant_docs = self.retriever.retrieve_prioritized_documents(
                question, self.vectorstore, k=8
            )

            if not relevant_docs:
                return """
This information is not available in the provided legal documents.
Please consult with a qualified legal advisor.
"""

            # Prepare enhanced context with priority information
            context = ""
            sources = {}

            for doc in relevant_docs:
                source = doc.metadata.get('source', 'Unknown')
                priority = self.retriever.get_document_priority(source)

                context += f"[Priority {priority}] Document: {source}\n"
                context += f"Content: {doc.page_content}\n\n"

                if source not in sources:
                    sources[source] = priority

            # Check if Nepali was requested explicitly
            nepali_requested = (
                "नेपाली" in question.lower() or "in nepali" in question.lower()
            )

            # Enhanced prompt with dynamic language rules
            if nepali_requested:
                enhanced_prompt = f"""
You are an expert Nepali legal assistant with deep knowledge of Nepal's legal system.

STRICT GUIDELINES:
1. Answer ONLY based on the provided Nepali legal documents
2. Prioritize information from higher priority documents (lower priority numbers)
3. If information conflicts between documents, prefer higher priority sources
4. If information is not in documents, state: "यो जानकारी उपलब्ध दस्तावेजहरूमा छैन।"
5. Always cite specific laws, acts, and sections with exact names
6. Provide comprehensive answers covering all relevant aspects
7. Provide answers in English (with Nepali sources in Nepali)

Legal Categories Identified: {', '.join(categories)}

Context from Nepali Legal Documents (Ordered by Priority):
{context}

Question: {question}

Provide a comprehensive answer including:

**कानुनी जानकारी (Legal Information in Nepali):**
[Detailed answer in Nepali with specific legal citations]

**Legal Information (English Summary):**
[Key points in English]

**सम्बन्धित कानुनहरू (Relevant Laws):**
[List acts, sections, and articles]

**आवश्यक कागजातहरू (Required Documents):**
[List if applicable]

**कानुनी प्रक्रिया (Legal Procedure):**
[Step-by-step]

**महत्वपूर्ण सुझावहरू (Important Notes):**
[Warnings & advice]

Answer:"""
            else:
                enhanced_prompt = f"""
You are an expert Nepali legal assistant with deep knowledge of Nepal's legal system.

STRICT GUIDELINES:
1. Answer ONLY based on the provided Nepali legal documents
2. Prioritize information from higher priority documents (lower priority numbers)
3. If information conflicts between documents, prefer higher priority sources
4. If information is not in documents, state: "This information is not available in the provided documents."
5. Always cite specific laws, acts, and sections with exact names
6. Provide comprehensive answers covering all relevant aspects
7. Respond ONLY in English (unless Nepali is explicitly requested)

Legal Categories Identified: {', '.join(categories)}

Context from Nepali Legal Documents (Ordered by Priority):
{context}

Question: {question}

Provide a comprehensive answer including:

**Legal Information:**
[Detailed answer in English with specific legal citations]

**Relevant Laws:**
[List acts, sections, and articles]

**Required Documents:**
[List if applicable]

**Legal Procedure:**
[Step-by-step]

**Important Notes:**
[Warnings & advice]

Answer:"""

            # Invoke LLM
            response = self.llm.invoke(enhanced_prompt)

            # Prepare source information with priorities
            source_list = []
            for source, priority in sorted(sources.items(), key=lambda x: x[1]):
                priority_label = {1: "Constitution", 2: "Main Codes", 3: "Acts", 4: "Regulations"}.get(priority, "Others")
                source_list.append(f"{source} ({priority_label})")

            source_text = f"\n\n📚 Sources: {', '.join(source_list)}"

            # Comprehensive disclaimer
            disclaimer = f"""

⚠️ Important Legal Notice:
• This information is for educational and general guidance purposes only
• For actual legal advice and representation, please consult with a qualified lawyer
• Legal procedures can be complex and may vary based on individual circumstances

🏛️ Contact Government of Nepal | Supreme Court | District Court if needed
"""

            return response.content + source_text + disclaimer

        except Exception as e:
            return f"❌ Error processing query: {str(e)}"


In [10]:
def extract_text_from_pdf(pdf_path):
    """Extract text from PDF file"""
    try:
        with open(pdf_path, 'rb') as file:
            pdf_reader = PyPDF2.PdfReader(file)
            text = ""
            for page in pdf_reader.pages:
                text += page.extract_text() + "\n"
        return text
    except Exception as e:
        print(f"Error reading {pdf_path}: {str(e)}")
        return ""

In [11]:
def get_all_pdfs(folder_path):
    """Get all PDF files from directory and subdirectories"""
    pdf_files = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.lower().endswith(".pdf"):
                full_path = os.path.join(root, file)
                pdf_files.append(full_path)
                print(f"Found: {full_path}")  # Debug info
    return pdf_files

In [12]:
def preprocess_nepali_text(text):
    """Clean and preprocess Nepali text"""
    # Remove extra whitespaces
    text = re.sub(r'\s+', ' ', text)
    # Keep Nepali characters, English characters, numbers, and basic punctuation
    text = re.sub(r'[^\u0900-\u097F\u0020-\u007E\n]', ' ', text)
    return text.strip()

In [13]:
def create_documents_with_enhanced_metadata(pdf_files):
    """Process PDFs with enhanced metadata for prioritization"""
    documents = []
    retriever = EnhancedNepaliLegalRetriever()

    for i, pdf_path in enumerate(pdf_files):
        print(f"Processing {i+1}/{len(pdf_files)}: {os.path.basename(pdf_path)}")

        text = extract_text_from_pdf(pdf_path)
        if text.strip():
            cleaned_text = preprocess_nepali_text(text)

            # Enhanced metadata
            relative_path = os.path.relpath(pdf_path, DOCS_DIR)
            priority = retriever.get_document_priority(relative_path)

            doc = Document(
                page_content=cleaned_text,
                metadata={
                    "source": os.path.basename(pdf_path),
                    "file_path": pdf_path,
                    "relative_path": relative_path,
                    "priority": priority,
                    "document_type": "legal_document",
                    "processing_date": datetime.now().isoformat()
                }
            )
            documents.append(doc)

    return documents

In [14]:
def create_enhanced_vector_store(documents):
    """Create FAISS vector store with enhanced chunking"""
    # Enhanced text splitter for Nepali text
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=CHUNK_SIZE,
        chunk_overlap=CHUNK_OVERLAP,
        separators=["\n\n", "\n", "।", "।।", ".", " ", ""]
    )

    texts = text_splitter.split_documents(documents)
    print(f"Created {len(texts)} text chunks")

    # Add priority information to chunk metadata
    for chunk in texts:
        if 'priority' not in chunk.metadata:
            retriever = EnhancedNepaliLegalRetriever()
            chunk.metadata['priority'] = retriever.get_document_priority(
                chunk.metadata.get('source', '')
            )

    # Create vector store
    vectorstore = FAISS.from_documents(texts, embeddings)
    vectorstore.save_local("enhanced_nepali_legal_vectorstore")
    print("✅ Enhanced vector store created and saved!")

    return vectorstore

In [26]:
class ComprehensiveNepaliLegalAssistant:
    def __init__(self, vectorstore, llm):
        self.vectorstore = vectorstore
        self.llm = llm
        self.retriever = EnhancedNepaliLegalRetriever()

        # Single English prompt that works with Nepali legal sources
        self.prompt = """
You are an expert legal assistant specializing in Nepali law. Your role is to provide accurate legal information based on Nepali legal documents.

Important Guidelines:
1. Only provide answers based on the provided Nepali legal documents (sources may be in Nepali language)
2. Always respond in English, even though the source documents are in Nepali
3. If information is not available in the documents, clearly state: "This information is not available in the provided documents."
4. Prioritize higher priority documents (Constitution > Main Codes > Acts > Regulations)
5. Always cite specific laws, acts, or sections
6. Be precise and do not make assumptions
7. Translate and interpret Nepali legal terms accurately into English

Identified legal categories: {categories}

Reference to Nepali legal documents (organized by priority):
{context}

Question: {question}

Please provide a detailed response in English in the following format:

**Main Legal Information:**
[Main answer in English, interpreting Nepali legal sources]

**Related Laws:**
[Specific acts, sections and articles with English translations]

**Required Procedures:**
[Step-by-step process if applicable]

**Important Considerations:**
[Important legal considerations and warnings]

Response:"""

    def get_response(self, question: str) -> str:
        """Get comprehensive legal response in English only"""
        if not self.vectorstore:
            return "❌ Legal documents are not loaded."

        try:
            # Identify categories
            categories = self.retriever.identify_legal_categories(question)

            # Get prioritized documents
            relevant_docs = self.retriever.retrieve_prioritized_documents(
                question, self.vectorstore, k=8
            )

            if not relevant_docs:
                return """
This information is not available in the provided legal documents.

Please consult with a qualified legal advisor for specific legal advice.
"""

            # Build context with priority information
            context = ""

            for doc in relevant_docs:
                source = doc.metadata.get('source', 'Unknown')
                priority = doc.metadata.get('priority', 5)

                # English priority labels only
                priority_labels = {
                    1: "Constitution",
                    2: "Main Code",
                    3: "Act",
                    4: "Regulation",
                    5: "Other"
                }

                priority_label = priority_labels.get(priority, "Other")

                context += f"[{priority_label}] {source}:\n{doc.page_content}\n\n"

            # Use single English prompt
            prompt = self.prompt.format(
                categories=', '.join(categories),
                context=context,
                question=question
            )

            response = self.llm.invoke(prompt)

            # Simplified disclaimer
            disclaimer = """

Legal procedures may vary based on individual circumstances.
For actual legal advice and representation, consult a qualified lawyer.
"""

            return response.content + disclaimer

        except Exception as e:
            return f"❌ Error: {str(e)}"

    # Remove separate language methods - only one method needed
    def get_legal_advice(self, question: str) -> str:
        """Get legal advice in English based on Nepali legal sources"""
        return self.get_response(question)

In [27]:
print("🔍 Loading PDF files...")
pdf_files = get_all_pdfs(DOCS_DIR)
print(f"📄 Found {len(pdf_files)} PDF files")

if pdf_files:
    print("⚙️ Processing documents with enhanced metadata...")
    documents = create_documents_with_enhanced_metadata(pdf_files)
    print(f"✅ Successfully processed {len(documents)} documents")

    print("🔧 Creating enhanced vector store...")
    vectorstore = create_enhanced_vector_store(documents)

    # Initialize LLM
    llm = ChatGroq(
        model_name=MODEL_NAME,
        temperature=0.1,
        max_tokens=3072
    )

    # Initialize enhanced assistant
    legal_assistant = ComprehensiveNepaliLegalAssistant(vectorstore, llm)
    print("✅ Comprehensive Nepali Legal Assistant ready!")

else:
    print("❌ No PDF files found")
    vectorstore = None
    legal_assistant = None

🔍 Loading PDF files...
Found: /content/drive/MyDrive/Legal/Constitution.pdf
Found: /content/drive/MyDrive/Legal/Adtional-Acts/केही_नेपाल_ऐनको_व्यवस्था_जगाउने_ऐन,_२०६३.pdf
Found: /content/drive/MyDrive/Legal/Adtional-Acts/मुलुकी_फौजदारी_कार्यविधि_संहिता,_२०७४.pdf
Found: /content/drive/MyDrive/Legal/Adtional-Acts/मुलुकी_देवानी_कार्यविधि_संहिता,_२०७४.pdf
Found: /content/drive/MyDrive/Legal/Adtional-Acts/मुलुकी_देवानी_संहिता,_२०७४.pdf
Found: /content/drive/MyDrive/Legal/Adtional-Acts/मुलुकी_अपराध_संहिता,_२०७४.pdf
Found: /content/drive/MyDrive/Legal/Adtional-Acts/विनियोजन_ऐन,_२०८२.pdf
Found: /content/drive/MyDrive/Legal/Adtional-Acts/राष्‍ट्र_ऋण_उठाउने_ऐन,_२०८२.pdf
Found: /content/drive/MyDrive/Legal/Adtional-Acts/आर्थिक_ऐन,_२०८२.pdf
Found: /content/drive/MyDrive/Legal/Adtional-Acts/शहीद_दशरथ_चन्द_स्वास्थ्य_विज्ञान_विश्‍वविद्यालय_ऐन,_२०८२.pdf
Found: /content/drive/MyDrive/Legal/Adtional-Acts/विद्युतिय_व्यापार_(इ-कमर्स)_ऐन,_२०८१.pdf
Found: /content/drive/MyDrive/Legal/Adtional-Acts/विधायन_ऐन

In [36]:
def enhanced_chatbot_interface(message, history):
    """Enhanced Gradio chatbot interface"""
    if not legal_assistant:
        return "❌ Legal assistant not available. Please ensure all documents are processed correctly."

    response = legal_assistant.get_response(message)
    return response

# Comprehensive examples
comprehensive_examples = [
    # Divorce related
    "I want to divorce my husband. What do I need to do legally?",
    "म मेरो श्रीमान्लाई तलाक दिन चाहन्छु। कानुनी रूपमा मैले के गर्नुपर्छ?",
    "What are the grounds for divorce in Nepal according to law?",

    # Child custody
    "If I want custody of my child, what are my rights?",
    "मेरो बच्चाको हिरासतको लागि मेरा अधिकारहरू के हुन्?",
    "How is child custody decided in divorce cases?",

    # Financial support
    "My husband is not giving me financial support, what can I do?",
    "मेरो श्रीमान्ले मलाई आर्थिक सहायता दिइरहेको छैन, म के गर्न सक्छु?",
    "What is the law regarding alimony in Nepal?",

    # Domestic violence
    "How can I file a complaint for domestic violence?",
    "घरेलु हिंसाको उजुरी कसरी दिन सकिन्छ?",
    "What protection does law provide for domestic violence victims?",

    # Property and inheritance
    "What are my property rights as a wife in Nepal?",
    "पत्नीको हैसियतमा मेरा सम्पत्ति अधिकारहरू के हुन्?",
    "How does inheritance law work for women in Nepal?",

    # Marriage laws
    "What are the legal requirements for marriage in Nepal?",
    "नेपालमा विवाहका लागि कानुनी आवश्यकताहरू के हुन्?"
]

# Enhanced demo interface
enhanced_demo = gr.ChatInterface(
    fn=enhanced_chatbot_interface,
    title="🏛️ Enhanced Nepali Legal Assistant | उन्नत नेपाली कानुनी सहायक",
    description="""
    <div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); padding: 30px; border-radius: 15px; color: white; text-align: center; margin-bottom: 20px;">
        <h2 style="margin: 0; color: white;">🏛️ Comprehensive Nepali Legal Information System</h2>
        <p style="margin: 10px 0; font-size: 18px;"><strong>नेपालको संविधान, ऐन र कानुनबाट आधिकारिक जानकारी</strong></p>
        <p style="margin: 0;"><em>Authoritative information from Nepal's Constitution, Acts, and Laws</em></p>
    </div>

    <div style="display: grid; grid-template-columns: 1fr 1fr; gap: 20px; margin: 20px 0;">
        <div style="background-color: #f8f9fa; padding: 20px; border-radius: 10px; border-left: 5px solid #28a745;">
            <h4 style="color: #28a745; margin-top: 0;">🔍 Document Priority System</h4>
            <ol style="margin: 0;">
                <li><strong>संविधान (Constitution)</strong> - Highest Priority</li>
                <li><strong>मुख्य संहिताहरू (Core Codes)</strong> - Civil, Criminal</li>
                <li><strong>ऐनहरू (Acts)</strong> - Specific legislation</li>
                <li><strong>नियमावली (Rules)</strong> - Regulatory documents</li>
            </ol>
        </div>
    </div>

    <div style="background-color: #fff3cd; padding: 15px; border-radius: 10px; border: 1px solid #ffeaa7; margin: 20px 0;">
        <h4 style="color: #856404; margin-top: 0;">⚠️ महत्वपूर्ण कानुनी सूचना (Important Legal Notice)</h4>
        <p style="margin: 5px 0;"><strong>नेपाली:</strong> यो प्रणाली केवल शैक्षिक जानकारी प्रदान गर्छ। वास्तविक कानुनी सल्लाहको लागि योग्य वकिलसँग सम्पर्क गर्नुहोस्।</p>
        <p style="margin: 5px 0;"><strong>English:</strong> This system provides educational information only. For actual legal advice, consult a qualified lawyer.</p>
    </div>
    """,
    examples=comprehensive_examples,
    cache_examples=False,
    theme=gr.themes.Soft(
        primary_hue="blue",
        secondary_hue="gray",
        font=gr.themes.GoogleFont("Noto Sans")
    ),
    css="""
    .gradio-container {
        max-width: 1000px !important;
        margin: auto !important;
    }
    .chat-message {
        font-size: 14px !important;
        line-height: 1.5 !important;
    }
.message.user {
    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%) !important;
    color: white !important;
}
.message.bot {
    border: 1px solid #e0e0e0 !important;
    border-left: 5px solid #28a745 !important;
    """
)

/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:345: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


In [37]:
def run_priority_test():
    """Test the priority system"""
    print("🧪 Testing Document Priority System")
    print("=" * 50)

    if not legal_assistant:
        print("❌ System not ready")
        return

    test_query = "विवाह र तलाकका कानुनहरू के हुन्?"

    print(f"Test Query: {test_query}")
    print("-" * 30)

    # Get prioritized documents
    docs = legal_assistant.retriever.retrieve_prioritized_documents(
        test_query, legal_assistant.vectorstore, k=5
    )

    print("Retrieved Documents (by priority):")
    for i, doc in enumerate(docs, 1):
        source = doc.metadata.get('source', 'Unknown')
        priority = doc.metadata.get('priority', 5)
        priority_labels = {1: "संविधान", 2: "मुख्य संहिता", 3: "ऐन", 4: "नियमावली", 5: "अन्य"}
        priority_label = priority_labels.get(priority, "अन्य")
        print(f"{i}. [{priority_label}] {source}")

    print(f"\nTotal documents retrieved: {len(docs)}")

In [38]:
def run_comprehensive_system_test():
    """Run comprehensive test of the entire system"""
    print("🚀 Running Comprehensive System Test")
    print("=" * 60)

    if not legal_assistant:
        print("❌ System not ready")
        return

    test_cases = [
        {
            "question": "I want to divorce my husband. What do I need to do legally?",
            "expected_categories": ["divorce"],
            "description": "English divorce query"
        },
        {
            "question": "म मेरो बच्चाको हिरासत चाहन्छु। मेरा अधिकारहरू के हुन्?",
            "expected_categories": ["custody"],
            "description": "Nepali custody query"
        },
        {
            "question": "घरेलु हिंसाको मामिलामा कानुनले के व्यवस्था गरेको छ?",
            "expected_categories": ["domestic_violence"],
            "description": "Domestic violence law query"
        },
        {
            "question": "विवाह र तलाक दुवैको बारेमा जानकारी चाहिन्छ।",
            "expected_categories": ["marriage", "divorce"],
            "description": "Multi-category query"
        }
    ]

    for i, test_case in enumerate(test_cases, 1):
        print(f"\n{i}. {test_case['description']}")
        print(f"Question: {test_case['question']}")
        print("-" * 50)

        # Test category identification
        identified_categories = legal_assistant.retriever.identify_legal_categories(test_case['question'])
        print(f"Identified Categories: {identified_categories}")

        # Test document retrieval
        docs = legal_assistant.retriever.retrieve_prioritized_documents(
            test_case['question'], legal_assistant.vectorstore, k=3
        )
        print(f"Retrieved Documents: {len(docs)}")

        if docs:
            print("Top 3 Documents by Priority:")
            for j, doc in enumerate(docs[:3], 1):
                source = doc.metadata.get('source', 'Unknown')
                priority = doc.metadata.get('priority', 5)
                print(f"  {j}. Priority {priority}: {source}")

        # Test full response
        try:
            response = legal_assistant.get_response(test_case['question'])
            print(f"✅ Response generated successfully ({len(response)} chars)")
        except Exception as e:
            print(f"❌ Response generation failed: {str(e)}")

        print("=" * 60)

In [39]:
def check_enhanced_system_status():
    """Check enhanced system status"""
    print("🔍 Enhanced System Status Check")
    print("-" * 40)

    # Check documents
    pdf_files = get_all_pdfs(DOCS_DIR)
    print(f"📄 PDF files found: {len(pdf_files)}")

    if pdf_files:
        print("📁 Document structure analysis:")
        folders = {}
        for pdf in pdf_files:
            folder = os.path.dirname(os.path.relpath(pdf, DOCS_DIR))
            if folder not in folders:
                folders[folder] = 0
            folders[folder] += 1

        for folder, count in folders.items():
            print(f"  📂 {folder}: {count} files")

    # Check vector store
    if vectorstore:
        print("✅ Enhanced vector store: Ready")
        print(f"📊 Total vectors: {vectorstore.index.ntotal}")

        # Test embedding model
        try:
            test_embedding = embeddings.embed_query("विवाह कानून")
            print(f"✅ Enhanced embeddings: Ready (dimension: {len(test_embedding)})")
        except Exception as e:
            print(f"❌ Embeddings error: {str(e)}")
    else:
        print("❌ Vector store: Not ready")

    # Check LLM
    if 'llm' in globals():
        try:
            test_response = llm.invoke("Test connectivity")
            print("✅ LLM (Groq): Connected and responsive")
        except Exception as e:
            print(f"❌ LLM Error: {str(e)}")
    else:
        print("❌ LLM: Not initialized")

    # Check legal assistant
    if legal_assistant:
        print("✅ Enhanced Legal Assistant: Ready")
        print("🎯 System fully operational!")
    else:
        print("❌ Legal Assistant: Not ready")

In [40]:
def demo_query_examples():
    """Demonstrate system with example queries"""
    if not legal_assistant:
        print("❌ System not ready for demo")
        return

    print("🎭 Demonstration of Enhanced Nepali Legal Assistant")
    print("=" * 60)

    demo_questions = [
        "नेपालको संविधानमा महिलाका अधिकारहरू के छन्?",
        "How can I file for divorce in Nepal?",
        "बच्चाको हिरासतका लागि अदालतमा कसरी निवेदन दिने?"
    ]

    for i, question in enumerate(demo_questions, 1):
        print(f"\n🔸 Demo {i}: {question}")
        print("-" * 50)

        try:
            response = legal_assistant.get_response(question)
            # Show first 300 characters of response
            preview = response[:300] + "..." if len(response) > 300 else response
            print(f"Response Preview:\n{preview}")
            print(f"\nFull response length: {len(response)} characters")
        except Exception as e:
            print(f"❌ Error: {str(e)}")

        print("=" * 60)


In [41]:
def launch_enhanced_chatbot():
    """Launch the enhanced chatbot interface"""
    if legal_assistant:
        print("🚀 Launching Enhanced Nepali Legal Assistant...")
        print("🌐 Creating public interface...")
        print("📱 Interface will be available at the provided URL")

        enhanced_demo.launch(
            share=True,
            debug=True,
            server_name="0.0.0.0",
            server_port=7860,
            show_error=True
        )
    else:
        print("❌ Cannot launch - system not ready")
        print("Please ensure all previous steps completed successfully")

In [42]:
check_enhanced_system_status()

🔍 Enhanced System Status Check
----------------------------------------
Found: /content/drive/MyDrive/Legal/Constitution.pdf
Found: /content/drive/MyDrive/Legal/Adtional-Acts/केही_नेपाल_ऐनको_व्यवस्था_जगाउने_ऐन,_२०६३.pdf
Found: /content/drive/MyDrive/Legal/Adtional-Acts/मुलुकी_फौजदारी_कार्यविधि_संहिता,_२०७४.pdf
Found: /content/drive/MyDrive/Legal/Adtional-Acts/मुलुकी_देवानी_कार्यविधि_संहिता,_२०७४.pdf
Found: /content/drive/MyDrive/Legal/Adtional-Acts/मुलुकी_देवानी_संहिता,_२०७४.pdf
Found: /content/drive/MyDrive/Legal/Adtional-Acts/मुलुकी_अपराध_संहिता,_२०७४.pdf
Found: /content/drive/MyDrive/Legal/Adtional-Acts/विनियोजन_ऐन,_२०८२.pdf
Found: /content/drive/MyDrive/Legal/Adtional-Acts/राष्‍ट्र_ऋण_उठाउने_ऐन,_२०८२.pdf
Found: /content/drive/MyDrive/Legal/Adtional-Acts/आर्थिक_ऐन,_२०८२.pdf
Found: /content/drive/MyDrive/Legal/Adtional-Acts/शहीद_दशरथ_चन्द_स्वास्थ्य_विज्ञान_विश्‍वविद्यालय_ऐन,_२०८२.pdf
Found: /content/drive/MyDrive/Legal/Adtional-Acts/विद्युतिय_व्यापार_(इ-कमर्स)_ऐन,_२०८१.pdf
Found: /co

In [43]:
launch_enhanced_chatbot()

🚀 Launching Enhanced Nepali Legal Assistant...
🌐 Creating public interface...
📱 Interface will be available at the provided URL
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://f1e60b71f461dde57d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 0.0.0.0:7860 <> https://f1e60b71f461dde57d.gradio.live
